In [1]:
from MetaLearner import *

In [1]:
import pickle
import numpy as np
import time
import jsonpickle

In [ ]:
# todo: https://stackoverflow.com/questions/24168569/how-to-add-names-to-a-numpy-array-without-changing-its-dimension

In [ ]:
# This file runs the experiment

In [13]:
tf.keras.backend.set_floatx('float64')

In [14]:
tf.keras.utils.set_random_seed(8953)

In [105]:
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic_one_run.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

In [122]:
# load with jsonpickle
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_all.json"
f = open(file_name, 'r')
json_str = f.read()
data = jsonpickle.decode(json_str)

In [5]:
n_setups = 1
sample_sizes = [500, 1000, 2000, 5000]
n_runs = 1

In [ ]:
learners = [TLearner('rf'), SLearner('rf'), XLearner('rf'), RLearner('rf'), DRLearner('rf'), RALearner('rf'),
            PWLearner('rf'), ULearner('rf'),
            TLearner('lasso'), SLearner('lasso'), XLearner('lasso'), RLearner('lasso'), DRLearner('lasso'),
            RALearner('lasso'), PWLearner('lasso'), ULearner('lasso'),
            TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]

In [7]:
learners = [TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]

In [ ]:
learners = [RLearner('nn')]

In [ ]:
# What we want: For each setting and base learner --> matrix which includes the MSE, columns: Metalearners + SampleSize, rows: Run(1-10)
# --> So we can plot the mean and 1se of the MSE for each Metalearner, each samplesize; giving the setting and baselearner
# --> number of plots: settings*baselearner.

In [14]:
results = []

for i in range(n_setups):
    results.append([])

for i in range(n_setups):
    for baselearner in range(3):
        results[i].append([])

results

[[[], [], []]]

In [18]:
results[0][0]

[]

In [9]:
dim = data[0][0][0][0].shape[1]  # dimension of x


# helper function to get y, x, w, tau, out of data[][][][]
def get_variables(dataset, setup, samplesize, run, train_test):
    y = dataset[setup][samplesize][run][train_test][:, 0]
    x = dataset[setup][samplesize][run][train_test][:, 1:(dim - 2)]
    w = dataset[setup][samplesize][run][train_test][:, (dim - 2)]
    tau = dataset[setup][samplesize][run][train_test][:, (dim - 1)]
    return y, x, w, tau

In [10]:
b = 0
m = 0
s = 0

for i in range(n_setups):
    print(f'Setup: {i + 1}')
    # array for all mses for one setup
    setup_mse = np.empty(shape=(0, 24))
    s = 0  # restart index for samplesize
    for size in sample_sizes:
        print(f'Sample Size: {s + 1}')
        # array for all mses for one setup and samplesize.
        size_mse = np.empty(shape=(0, 24))
        for r in range(n_runs):
            print(f'Run: {r + 1}')
            print('-------------')
            # array for all mses in one setup, samplesize and run.
            mses = np.empty(shape=(1, 24))
            # get data for specific setup, samplesize and run.
            temp_y_train, temp_x_train, temp_w_train, temp_tau_train = get_variables(dataset=data, setup=i,
                                                                                     samplesize=s, run=r,
                                                                                     train_test=0)
            temp_y_test, temp_x_test, temp_w_test, temp_tau_test = get_variables(dataset=data, setup=i,
                                                                                 samplesize=s, run=r,
                                                                                 train_test=1)
            # restart index for metalearner
            m = 0
            for l in learners:
                if m % 8 == 0:
                    print('---')
                    print(f'BaseLearner: {int((m / 8) + 1)}')
                print(f'Learner {m + 1}: {l.name}')
                tic = time.time()
                # training and testing MetaLearner.
                learner = l
                learner.fit(temp_x_train, temp_y_train, temp_w_train)
                predictions = learner.predict(temp_x_test)
                temp_mse = ((predictions - temp_tau_test) ** 2).mean()
                # append mse of specific metalearner to 'mses'.
                mses[0, m] = temp_mse
                # print time
                toc = time.time()
                print(f'Time: {round(toc - tic, 4)} seconds.')
                # update index
                m += 1
            # append 'mses' to 'size_mse'.
            size_mse = np.append(size_mse, mses, axis=0)
        # append 'size_mse' to 'setup_mse'.
        setup_mse = np.append(setup_mse, size_mse, axis=0)
        # update index
        s += 1
    # append to results
    results[i][0] = setup_mse[:, 0:8]  # random forest
    #results[i][1] = setup_mse[:, 8:16]  # lasso
    #results[i][2] = setup_mse[:, 16:24]  # neural network





# this should make sense i guess! :)
# TODO: CHECK IF IT MAKES IT FASTER IF EVERYTHING IS EITHER IN NP OR IN LIST/ARRAY BASE PYTHON.
# TODO: DONE :)
# TODO: DONE :)
# TODO: DONE :)

### IT TOOK 6M 40S FOR --> SAMPLE SIZES [200, 200], TEST SIZE = 50, 2 SETUPS, 2 RUNS LOL

### 35min for all Baselearners, 1 Setup, 1 Run, 4 Sample Sizes  ()

Setup: 1
Sample Size: 1
Run: 1
-------------
---
BaseLearner: 1
Learner 1: TLearner
Time: 3.8361 seconds.
Learner 2: SLearner
Time: 1.615 seconds.
Learner 3: XLearner
Time: 6.4887 seconds.
Learner 4: RLearner
Time: 4.3666 seconds.
Learner 5: DRLearner
Time: 51.4826 seconds.
Learner 6: RALearner
Time: 5.8176 seconds.
Learner 7: PWLearner
Time: 2.691 seconds.
Learner 8: ULearner
Time: 4.9662 seconds.
Sample Size: 2
Run: 1
-------------
---
BaseLearner: 1
Learner 1: TLearner
Time: 5.951 seconds.
Learner 2: SLearner
Time: 3.1454 seconds.
Learner 3: XLearner
Time: 10.4338 seconds.
Learner 4: RLearner
Time: 6.2985 seconds.
Learner 5: DRLearner
Time: 18.9648 seconds.
Learner 6: RALearner
Time: 5.5957 seconds.
Learner 7: PWLearner
Time: 3.1783 seconds.
Learner 8: ULearner
Time: 5.4885 seconds.
Sample Size: 3
Run: 1
-------------
---
BaseLearner: 1
Learner 1: TLearner
Time: 7.0587 seconds.
Learner 2: SLearner
Time: 4.2848 seconds.
Learner 3: XLearner
Time: 15.7597 seconds.
Learner 4: RLearner
T

In [ ]:
### Training tau0 and tau1 takes extremely long!! compared to mu0 and mu1 in lasso!!! probably because of the max_iter

In [ ]:
setup_mse

In [ ]:
# set n_trees to 1000 to gain time or maybe even 500

In [31]:
results[0][0]

array([[8.82242677e-01, 1.34512945e-01, 4.13139923e-01, 3.46823828e-01,
        1.72831295e-01, 5.51353617e-01, 6.00251483e+00, 1.04135023e+00],
       [8.21603347e-01, 2.02012140e-01, 4.50496680e-01, 1.79659982e+00,
        9.39436409e+01, 7.24155584e-01, 2.98456305e+01, 1.04056304e+01],
       [6.90702094e-01, 1.56491740e-01, 4.91725492e-01, 1.77548610e+00,
        2.96085594e+00, 6.09572286e-01, 3.72494029e+01, 1.95863895e+00],
       [3.35695025e-01, 4.81432903e-02, 2.13545315e-01, 6.87465220e-01,
        7.27462659e+00, 3.46110284e-01, 8.90549421e+01, 1.89862817e+00]])

In [33]:
# file name
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/results.json"

# SAVE LIST AS JSON FILE
f = open(file_name, 'w')
json_obj = jsonpickle.encode(results)
f.write(json_obj)
f.close()

In [34]:
# open
# LOAD JSON FILE
f = open(file_name, 'r')
json_str = f.read()
results_load = jsonpickle.decode(json_str)

In [41]:
results_load[0][0]

array([[8.82242677e-01, 1.34512945e-01, 4.13139923e-01, 3.46823828e-01,
        1.72831295e-01, 5.51353617e-01, 6.00251483e+00, 1.04135023e+00],
       [8.21603347e-01, 2.02012140e-01, 4.50496680e-01, 1.79659982e+00,
        9.39436409e+01, 7.24155584e-01, 2.98456305e+01, 1.04056304e+01],
       [6.90702094e-01, 1.56491740e-01, 4.91725492e-01, 1.77548610e+00,
        2.96085594e+00, 6.09572286e-01, 3.72494029e+01, 1.95863895e+00],
       [3.35695025e-01, 4.81432903e-02, 2.13545315e-01, 6.87465220e-01,
        7.27462659e+00, 3.46110284e-01, 8.90549421e+01, 1.89862817e+00]])

In [ ]:
# 0.885576,0.134742,0.411388,0.344136,0.128143,0.588750,5.880639,1.026057
# 0.815042,0.201147,0.439917,1.577994,10.549566,0.710207,29.923138,10.357554
# 0.679716,0.154199,0.399313,1.942728,0.317815,0.607700,37.089789,1.663813
# 0.285524,0.053260,0.222201,0.769516,6.082601,0.306323,92.644239,2.648658

# PW gets worse with increasing sample size, lol

In [ ]:
# results[0][2]
# 0.694569,0.167760,0.567150,2.997832,0.467543,0.944197,16.717111,12.709246
# 0.610678,0.187881,0.759497,22.874956,6.021213,1.137736,58.716848,1734.143168
# 0.864565,0.183118,0.793892,7.919859,10.800849,1.035885,72.427161,343.769983
# 0.233476,0.074466,0.366872,3.135558,34.819663,0.473895,124.064119,1850.331233

# --> PW and U-learner are bad here (why?)
# --> looks nice otherwise :)

In [ ]:
# Results[0][1]
# 2.308568,1.004442,1.526483,0.028155,2.156328,2.115060,0.036090,0.033795
# 1.791405,1.193947,1.260019,0.044737,0.323602,1.588854,5.308641,0.042311
# 0.315428,0.336822,0.348231,0.023158,0.071988,0.244252,0.000031,0.017461
# 0.023137,0.060074,0.034748,0.000366,0.000355,0.008674,0.225957,0.000360

# --> nice :)

In [ ]:
# Results[0][0]
# 2.164445,0.004278,0.921012,3.660252,1.532306,1.524151,4.093397,4.733766
# 3.453443,0.004395,0.898049,4.667126,1.998041,2.072289,11.121975,5.068848
# 2.233662,0.006844,0.710762,4.107797,1.644121,1.576521,14.268573,4.718986
# 0.385062,0.001257,0.100568,0.423951,0.210232,0.208753,16.626195,0.465520


# --> nice :)

In [ ]:
data[0][0][0][0]

In [ ]:
# https://amueller.github.io/aml/02-supervised-learning/06-linear-models-classification.html


In [3]:
i = 10
print(i)

10


In [ ]:
i = 1  # = setup number

# here comes the same loop

# change

# results[0] = ...
# results[1] = ...
# results[2] = ...


# save results.setupnumber.jsonpickle lol.

# --> it means 24 files :)

Results from all on eth cluster

In [2]:
# open
# LOAD JSON FILE
file_name = "/Users/arberimbibaj/results.json"
f = open(file_name, 'r')
json_str = f.read()
results_all = jsonpickle.decode(json_str)

In [210]:
results_all[23][0]

array([[1.91420779e-01, 1.94469960e-02, 1.94350904e-01, 1.10410404e+03,
        1.87133219e-01, 1.84476011e-01, 1.10355854e-01, 3.47640137e+04],
       [4.21495760e-01, 1.41175229e+00, 1.04331432e+00, 3.49327398e+02,
        4.59807370e-01, 4.58598492e-01, 5.86746323e-01, 9.64299545e+03],
       [3.53130519e-01, 1.84869952e-01, 4.20593556e-01, 2.86513368e+02,
        3.71464544e-01, 3.73274726e-01, 1.64980995e-01, 6.97978561e+04],
       [1.69484118e-01, 1.19301257e-01, 2.26812348e-01, 3.69318065e+02,
        1.65059822e-01, 1.66206162e-01, 7.44889481e-01, 1.86449892e+04]])

In [ ]:
# S-Learner best, X_learner good, T, DR, RA, also good, only PW-bad
# almost same as one, PW also good?
# X-learner best, S-learner worst (PW actually worst, but extreme)
# X-learner best, DR/PW also good, rest oke (PW very bad)
# T-learner best?, R/RA very good
# X-learner best, but S/T/DR/RA all very good
# S-learner best, then X-learner
# S-learner best,
# X-learner best, (especially for RF)
# X-learner best,
# T-learner best
# S-learner best?, T/X/DR/RA all very good
# S-learner best, X also very good
# S-learner best, X also very good, R bad
# X-learner best, DR-learner also very good
# X-learner best, DR/RA good but then at siz=5000 bad?
# DR/RA best, X also very good
# S-learner best ??? probably because the cates are not scaled enough
# S-learner best
# S-learner best
# X best, DR/RA very good
# X best, DR/RA very good,
# X best, but all perform a bit bad
# T, S, X R DR all good, probably because of not scalling

In [ ]:
# CONCLUSIONS
# -----------

# 1: If zero cate --> S-learner is the best, X-learner also performs good
# 2: If simple cate --> X-learner is the best, often DR/RA/R learner also perform good
# 3: if complex cate --> T very good, somehow S-learner also good (probably to bad scalling of outcomes), DR/RA/R learner also good
# 4: PW learner very bad mostly, U-learner also not good

# Base learners
# -------------
# RF and Linear model seem to be more stable than Neural Network, at least with these hyperparameters.


In [ ]:
# All "pseudo-outome" metalearners very bad with Neural Network??? --> probably the classification task is bad

Data

In [130]:
data[5][0][0][0]

array([[ 8.34471335e-01, -5.28563234e-01, -2.80141251e-01, ...,
         6.81146622e-01,  0.00000000e+00,  1.49932657e-09],
       [ 1.81080934e-01,  6.49321509e-01, -5.33576639e-01, ...,
        -4.15951057e-01,  1.00000000e+00,  1.40798480e-05],
       [ 5.53730887e-01, -5.58128760e-03,  2.92908514e-01, ...,
        -1.90266595e-01,  0.00000000e+00,  7.11509053e-04],
       ...,
       [ 6.57050299e-03, -6.73184449e-01,  3.23333753e-01, ...,
         5.75459890e-01,  0.00000000e+00,  3.29637519e-07],
       [ 5.09824942e-02,  1.20463867e-02, -8.25670819e-01, ...,
         4.34569234e-01,  0.00000000e+00,  1.40946009e-09],
       [ 7.15493657e-01,  2.74820625e-01, -1.06849057e+00, ...,
         1.56119488e-01,  1.00000000e+00,  1.68305846e-09]])